In [2]:
import warnings; warnings.filterwarnings('ignore')

In [3]:
from growing_instability_lib import *

Using TensorFlow backend.


In [4]:
sample_sub = pd.read_csv('../data/sampleSubmission.csv')
topics = sorted(set(sample_sub.columns.difference(['id'])))

topic2actual = {}
for i in sample_sub.columns:
    if 'id' == i:
        continue
    topic2actual[i] = segment(i)
    
target_columns = sorted(topics)
len(topics)

160

In [5]:
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.lsimodel import LsiModel
from gensim.corpora import Dictionary

dictionary = TfidfModel.load('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.dictionary')
tfidf = TfidfModel.load('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.tfidf')
lsi = LsiModel.load('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.lsi')

In [6]:
wvmodel = Word2Vec.load('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.w2v.model')

In [7]:
fsmodel = fasttext.load_model('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.fasttext.model.bin')

In [8]:
topic2actual

{'activism': ['activism'],
 'afghanistan': ['afghanistan'],
 'aid': ['aid'],
 'algerianhostagecrisis': ['algerian', 'hostage', 'crisis'],
 'alqaida': ['al', 'qaida'],
 'alshabaab': ['al', 'shaba', 'ab'],
 'antiwar': ['antiwar'],
 'arabandmiddleeastprotests': ['arab', 'and', 'middle', 'east', 'protests'],
 'armstrade': ['arms', 'trade'],
 'australianguncontrol': ['australian', 'gun', 'control'],
 'australiansecurityandcounterterrorism': ['australian',
  'security',
  'and',
  'counterterrorism'],
 'bastilledaytruckattack': ['bastille', 'day', 'truck', 'attack'],
 'belgium': ['belgium'],
 'berlinchristmasmarketattack': ['berlin', 'christmas', 'market', 'attack'],
 'bigdata': ['big', 'data'],
 'biometrics': ['biometrics'],
 'bokoharam': ['boko', 'haram'],
 'bostonmarathonbombing': ['boston', 'marathon', 'bombing'],
 'britisharmy': ['british', 'army'],
 'brusselsattacks': ['brussels', 'attacks'],
 'cameroon': ['cameroon'],
 'carers': ['carers'],
 'charliehebdoattack': ['charlie', 'hebdo', 

In [9]:
manual_topics = {
    'alshabaab': ['al', 'shabaab'],
#     'earthquakes': ['earthquake'],
#     'firefighters': ['firefighter'],
    'rio20earthsummit': ['rio', '20', 'earth', 'summit'],
    'zikavirus': ['zika']
}

topic2actual.update(manual_topics)

In [10]:
wvmodel.most_similar('shabaab')

[(u'qaida', 0.7521548271179199),
 (u'shabab', 0.750149667263031),
 (u'aqim', 0.7214016914367676),
 (u'islamist', 0.7179936170578003),
 (u'boko', 0.7109889388084412),
 (u'qaeda', 0.7079288363456726),
 (u'nusra', 0.6980888843536377),
 (u'taliban', 0.6857478022575378),
 (u'somalia', 0.6842936873435974),
 (u'militants', 0.6828135251998901)]

# Perform exploratory unsupervised discovery of unseen topics

In [11]:
%%time
# wvec_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'wvec_trainingX')
# fvec_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'fvec_trainingX')

# tfidf_wvec_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'tfidf_wvec_trainingX')
# tfidf_fvec_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'tfidf_fvec_trainingX')
# tfidf_lsi_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'tfidf_lsi_trainingX')

# word2idx_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'word2idx_trainingX')
_word2idx = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', '_word2idx')
trainingY = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'trainingY')

CPU times: user 1.44 s, sys: 516 ms, total: 1.96 s
Wall time: 2.41 s


In [12]:
training_topics_freq = trainingY.sum()
training_topics_freq.head()

activism                     NaN
afghanistan              10032.0
aid                       1891.0
algerianhostagecrisis       58.0
alqaida                   3194.0
dtype: float64

In [13]:
unseen_topics = set(topics).intersection(training_topics_freq.ix[training_topics_freq.isnull()].index)

In [14]:
unseen_topics

{'activism',
 'bastilledaytruckattack',
 'berlinchristmasmarketattack',
 'brusselsattacks',
 'charliehebdoattack',
 'francetrainattack',
 'munichshooting',
 'orlandoterrorattack',
 'parisattacks',
 'peaceandreconciliation',
 'sanbernardinoshooting',
 'tunisiaattack2015',
 'turkeycoupattempt',
 'zikavirus'}

In [15]:
ind2word = {j:i for i, j in _word2idx.iteritems()}
ind2class = dict(enumerate(topics))
class2ind = {j: i for i, j in ind2class.items()}

## Use idf only in tfidf weighted vectors

In [202]:
def transform_tfidf_word2vec(tokens, stopwords=[], trim=True):
#     global wvmodel
#     global tfidf
    # This requires wvmodel to be present in the namespace.
    wv_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords and w in wvmodel.wv.vocab)]
    )
    
    if trim:
        wv_feature_vec = wv_feature_vec.map(
        lambda x: filter(lambda s: s[1] > 0.1, sorted(tfidf[dictionary.doc2bow(x)], key=lambda y: y[1], reverse=True))
        )
    else:
        wv_feature_vec = wv_feature_vec.map(
        lambda x: tfidf[dictionary.doc2bow(x)]
        )
        
    wv_feature_vec = wv_feature_vec.map(
        lambda x: np.array([wvmodel[dictionary.id2token[id]] * w for id, w in x]).mean(axis=0) if len(x) > 0 else np.nan
    )

    return wv_feature_vec


def transform_tfidf_fasttext(tokens, stopwords=[], trim=True):
#     global fsmodel
#     global tfidf
    # This requires wvmodel to be present in the namespace.
    fs_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords)]
    )
    
    if trim:
        fs_feature_vec = fs_feature_vec.map(
        lambda x: filter(lambda s: s[1] > 0.1, sorted(tfidf[dictionary.doc2bow(x)], key=lambda y: y[1], reverse=True))
        )
    else:
        fs_feature_vec = fs_feature_vec.map(
        lambda x: tfidf[dictionary.doc2bow(x)]
        )
        
    fs_feature_vec = fs_feature_vec.map(
        lambda x: np.array([np.array(fsmodel[dictionary.id2token[id]]) * w for id, w in x]).mean(axis=0) if len(x) > 0 else np.nan
    )

    return fs_feature_vec


def build_lsi_vector(x, trim=True):
    v = np.zeros(lsi.num_topics)
    if trim:
        corpus = filter(lambda s: s[1] > 0.1, sorted(tfidf[dictionary.doc2bow(x)], key=lambda y: y[1], reverse=True))
    else:
        corpus = tfidf[dictionary.doc2bow(x)]

    for ix, vv in lsi[corpus]:
        v[ix] = vv

    return v


def transform_tfidf_lsi(tokens, stopwords=[], trim=True):
#     global fsmodel
#     global tfidf
    # This requires wvmodel to be present in the namespace.
    lsi_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords)]
    ).map(
        lambda x: build_lsi_vector(x, trim) if len(x) > 0 else np.nan
    )

    return lsi_feature_vec


def transform_fasttext(tokens, stopwords=[]):
    global fsmodel
    # This requires fsmodel to be present in the namespace.
    fs_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords)]
    ).map(lambda x: np.array([fsmodel[w] for w in x]).mean(axis=0) if len(x) > 0 else np.nan)

    return fs_feature_vec


def transform_word2vec(tokens, stopwords=[]):
    global wvmodel
    # This requires wvmodel to be present in the namespace.
    wv_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords and w in wvmodel.wv.vocab)]
    ).map(lambda x: np.array([wvmodel[w] for w in x]).mean(axis=0) if len(x) > 0 else np.nan)

    return wv_feature_vec


def parallel_generate_word_vectors(samp, transformer, stopwords, batch, num_proc):
    with Parallel(n_jobs=num_proc) as parallel:
        dataset = []
        is_break = False
        i = 0

        while not is_break:
            payload = []

            for j in xrange(num_proc):
                t_df = samp[(i + j) * batch: (i + 1 + j) * batch]

                if t_df.empty:
                    is_break = True
                    continue

                payload.append(
                    delayed(transformer)(
                        t_df, stopwords
                    )
                )

            print('Current batch in main thread: {}'.format((i + j) * batch))

            if payload:
                results = parallel(payload)
                dataset.extend(results)
                i += num_proc

    return pd.concat(dataset)

In [198]:
def word2idx_transform(word, _word2idx):
    return _word2idx.get(word, 0)

def extract_features_for(df, min_batch=2000, stopwords=[], num_proc=7):
    df_tokens = transform_text(df)
    
    batch = min(df_tokens.shape[0] / num_proc, min_batch)

    print('Computing fs features...')
    fvec = parallel_generate_word_vectors(df_tokens, transform_fasttext, stopwords=stopwords, batch=batch, num_proc=num_proc)

    print('Computing wv features...')
    wvec = parallel_generate_word_vectors(df_tokens, transform_word2vec, stopwords=stopwords, batch=batch, num_proc=num_proc)

    print('Mapping word indices...')
#     word_indices = []
    word_indices = df_tokens.map(lambda x: [word2idx_transform(i, _word2idx) for i in x.split()])

    print('Computing tfidf fs features...')
    tfidf_fvec = parallel_generate_word_vectors(df_tokens, transform_tfidf_fasttext, stopwords=stopwords, batch=batch, num_proc=num_proc)

    print('Computing tfidf wv features...')
    tfidf_wvec = parallel_generate_word_vectors(df_tokens, transform_tfidf_word2vec, stopwords=stopwords, batch=batch, num_proc=num_proc)

    print('Computing tfidf lsi features...')
    tfidf_lsi = parallel_generate_word_vectors(df_tokens, transform_tfidf_lsi, stopwords=stopwords, batch=batch, num_proc=num_proc)
    
    return word_indices, wvec, fvec, tfidf_wvec, tfidf_fvec, tfidf_lsi

In [64]:
with open('../data/TestData.json') as fl:
    data = json.load(fl)
    test_df = pd.DataFrame(data['TestData']).T
    del(data)

In [65]:
dictionary.id2token[855]

u'mean'

In [69]:
test_df.iloc[4031].bodyText

u'Eagles of Death Metal will return to Paris and Europe to complete their postponed tour dates, which were put on hold after the terrorist attack at the Bataclan theatre in November while the band was on stage. The group play the Olympia theatre in Paris on 16 February and will tour Europe throughout February and March. Eagles of Death Metal co-founder and frontman Jesse Hughes said that not returning to finish their set \u201cwas never an option\u201d and that he and the band have been overwhelmed by the response from Parisians. He added: \u201cThe people of Paris have always been incredible to us, and our feeling of love towards this beautiful city and its people has been reinforced a million times over this past month. \u201cWe look forward to coming back in February and continuing our mission to bring rock\u2019n\u2019roll to the world.\u201d In an interview with Vice, the band had vowed to return to Paris and also spoke about the attack on 13 November, during which 89 people were 

In [ ]:
'munichshooting': [array([6268, 6254, 6231, 6267, 6232]),
  array([4031, 6268, 6231, 2129, 6254]),
  array([6239, 6254, 6249, 6231, 6253])],

In [70]:
[(dictionary.id2token[i], j) for i, j in sorted(tfidf[dictionary.doc2bow(test_transformed_text.iloc[6239].split())], key=lambda x: x[1], reverse=True)]

[(u'gun', 0.47908648197935094),
 (u'firearms', 0.2621729058503798),
 (u'winnenden', 0.2381904582652168),
 (u'licenceholder', 0.15530949071104125),
 (u'kretschmerat', 0.15530949071104125),
 (u'psychological', 0.14207107160859914),
 (u'guns', 0.14149380242237303),
 (u'ownership', 0.13582370062322838),
 (u'emsdetten', 0.1314301443662301),
 (u'evaluation', 0.12106822170844278),
 (u'killings', 0.10994652147059226),
 (u'million', 0.10765912658638307),
 (u'erfurt', 0.1055917199333589),
 (u'automatic', 0.10469765726514944),
 (u'munich', 0.10396684514878873),
 (u'german', 0.10382804571058689),
 (u'germany', 0.1014591683608061),
 (u'legally', 0.10033452905366966),
 (u'acquisition', 0.09964990839489031),
 (u'safes', 0.09517284131025369),
 (u'controls', 0.09391237500199509),
 (u'regulations', 0.09354134610662412),
 (u'registered', 0.09172409244983849),
 (u'licence', 0.08797275674703604),
 (u'reforms', 0.08658069106129357),
 (u'glock', 0.08642191094617076),
 (u'9mm', 0.08498510781797099),
 (u'shoot

In [19]:
test_transformed_text = transform_text(test_df)

In [73]:
%%time
test_word_indices,test_wvec, test_fvec, test_tfidf_wvec, test_tfidf_fvec, test_tfidf_lsi = extract_features_for(
    test_df
)

Computing fs features...
Current batch in main thread: 6498
Current batch in main thread: 14079
Computing wv features...
Current batch in main thread: 6498
Current batch in main thread: 14079
Mapping word indices...
Computing tfidf fs features...
Current batch in main thread: 6498
Current batch in main thread: 14079
Computing tfidf wv features...
Current batch in main thread: 6498
Current batch in main thread: 14079
Computing tfidf lsi features...
Current batch in main thread: 6498
Current batch in main thread: 14079
CPU times: user 40.2 s, sys: 1.53 s, total: 41.7 s
Wall time: 1min 29s


In [203]:
def filter_stopwords(x, stopwords=['and', 'day', 'of', 'in']):
    return [i for i in x if i not in stopwords]

tpa = pd.DataFrame(topic2actual.items(), columns=['topics', 'actual'])

tpa['wv'] = transform_word2vec(tpa.actual.map(lambda x: ' '.join(filter_stopwords(x))))
tpa['fs'] = transform_fasttext(tpa.actual.map(lambda x: ' '.join(filter_stopwords(x))))

tpa['tk'] = tpa.actual.map(lambda x: [word2idx_transform(i, _word2idx) for i in x])

tpa['tfidf_wv'] = transform_tfidf_word2vec(tpa.actual.map(lambda x: ' '.join(filter_stopwords(x))), trim=False)
tpa['tfidf_fs'] = transform_tfidf_fasttext(tpa.actual.map(lambda x: ' '.join(filter_stopwords(x))), trim=False)
tpa['tfidf_lsi'] = transform_tfidf_lsi(tpa.actual.map(lambda x: ' '.join(filter_stopwords(x))), trim=False)

unseen_topics2vec = tpa[tpa.topics.isin(unseen_topics)]
unseen_topics2vec = unseen_topics2vec.set_index('topics')
unseen_topics2vec = unseen_topics2vec.ix[sorted(unseen_topics)]

all_topics2vec = tpa[tpa.topics.isin(topics)]
all_topics2vec = all_topics2vec.set_index('topics')
all_topics2vec = all_topics2vec.ix[sorted(topics)]

In [204]:
null_index = test_wvec[test_wvec.isnull()].index
if any(null_index):
    test_wvec.ix[null_index] = [[0] * wvmodel.vector_size] * 3
print(len(null_index))
    
null_index = test_fvec[test_fvec.isnull()].index
if any(null_index):
    test_fvec.ix[null_index] = [[0] * fsmodel.dim] * 3
print(len(null_index))

null_index = test_tfidf_wvec[test_tfidf_wvec.isnull()].index
if any(null_index):
    test_tfidf_wvec.ix[null_index] = [[0] * wvmodel.vector_size] * 3
print(len(null_index))
    
null_index = test_tfidf_fvec[test_tfidf_fvec.isnull()].index
if any(null_index):
    test_tfidf_fvec.ix[null_index] = [[0] * fsmodel.dim] * 3
print(len(null_index))
    
null_index = test_tfidf_lsi[test_tfidf_lsi.isnull()].index
if any(null_index):
    test_tfidf_lsi.ix[null_index] = [[0] * lsi.num_topics] * 3
print(len(null_index))


0
0
0
0
0


In [205]:
unsupervised_unseen_topic_detection = {
    'tfidf_wvec': [test_tfidf_wvec, unseen_topics2vec.tfidf_wv],
    'tfidf_fvec': [test_tfidf_fvec, unseen_topics2vec.tfidf_fs],
    'tfidf_lsi': [test_tfidf_lsi, unseen_topics2vec.tfidf_lsi],
#     'tk': 
}

unsupervised_all_topic_detection = {
    'tfidf_wvec': [test_tfidf_wvec, all_topics2vec.tfidf_wv],
    'tfidf_fvec': [test_tfidf_fvec, all_topics2vec.tfidf_fs],
    'tfidf_lsi': [test_tfidf_lsi, all_topics2vec.tfidf_lsi],
#     'tk': 
}

In [252]:
# N = 10
# unseen_topics_rank = {}

# for key, (tstvec, tpvec) in unsupervised_unseen_topic_detection.items():
#     sim = cosine_similarity(
#         np.vstack(tstvec),
#         np.vstack(tpvec),
#     )

#     for i, t in enumerate(unseen_topics2vec.index):
#         idx = sim[:, i].argsort()[::-1][:N]
#         v = sim[:, i][idx]

#         unseen_topics_rank.setdefault(t, []).append(idx)

In [215]:
N = 10
topics_rank = {}
test_max_class_df = pd.DataFrame()
test_max_score_df = pd.DataFrame()
test_max_class_score_df = pd.DataFrame()

for key, (tstvec, tpvec) in unsupervised_unseen_topic_detection.items():
    sim = cosine_similarity(
        np.vstack(tstvec),
        np.vstack(tpvec),
    )

#     for i, t in enumerate(all_topics2vec.index):
#         idx = sim[:, i].argsort()[::-1][:N]
#         v = sim[:, i][idx]

#         topics_rank.setdefault(t, []).append(idx)
    test_max_class_score_df[key] = pd.Series(zip(tpvec.index[sim.argmax(axis=1)], sim.max(axis=1)), index=tstvec.index)
    test_max_score_df[key] = pd.Series(sim.max(axis=1), index=tstvec.index)
    test_max_class_df[key] = pd.Series(tpvec.index[sim.argmax(axis=1)], index=tstvec.index)


In [207]:
# cosine_similarity(test_tfidf_wvec.iloc[4489], np.vstack(all_topics2vec.tfidf_wv))

In [208]:
sc = StandardScaler()

In [216]:
# test_max_score_df_normed = pd.DataFrame(sc.fit_transform(test_max_score_df), columns=test_max_score_df.columns, index=test_max_score_df.index)
test_max_score_df_normed = test_max_score_df

In [239]:
g = test_max_score_df_normed.max(axis=1)
g = g[g > 0.6]
w = test_max_class_df.ix[g.index]

In [240]:
g

TestData_00006    0.719141
TestData_00007    0.829392
TestData_00008    0.757273
TestData_00009    0.618126
TestData_00014    0.612950
TestData_00016    0.621031
TestData_00018    0.632733
TestData_00022    0.672172
TestData_00024    0.663084
TestData_00029    0.667131
TestData_00030    0.665910
TestData_00032    0.667570
TestData_00035    0.818714
TestData_00036    0.774036
TestData_00037    0.630042
TestData_00038    0.649132
TestData_00039    0.640741
TestData_00040    0.653708
TestData_00042    0.730308
TestData_00044    0.784936
TestData_00046    0.891608
TestData_00047    0.886960
TestData_00050    0.849932
TestData_00051    0.859087
TestData_00053    0.699846
TestData_00054    0.610185
TestData_00055    0.729341
TestData_00056    0.781060
TestData_00057    0.735029
TestData_00058    0.791485
                    ...   
TestData_07520    0.657297
TestData_07521    0.727538
TestData_07522    0.811195
TestData_07523    0.630735
TestData_07525    0.636060
TestData_07527    0.784651
T

In [326]:
from collections import Counter

In [339]:
sh = pd.DataFrame()
q = w[w.apply(lambda x: min(x) == max(x), 1)]
# q = w[w.apply(lambda x: len(set(x)) <= 2, 1)]
sh['class'] = q.apply(max, 1)
# sh['class'] = q.apply(lambda x: Counter(x).most_common()[0][0], 1)
sh['scores'] = g

In [340]:
sh = sh.sort_values(['class', 'scores'])
sh['order'] = sh.index.str.replace('TestData_', '').map(int)

In [243]:
for topic, group in sh.groupby('class'):
    group = group.sort_values(['scores'], ascending=False)[:10]
    print topic, group['order'].median(), group['order'].quantile(0.25), group['order'].quantile(0.75), group.shape[0]

bastilledaytruckattack 6140.5 6110.75 6167.75 10
berlinchristmasmarketattack 7514.5 7493.5 7537.75 10
brusselsattacks 5046.0 5027.5 5120.25 10
charliehebdoattack 659.5 129.75 1287.25 10
francetrainattack 2263.5 1086.5 2973.5 10
munichshooting 6235.0 5993.5 6251.25 10
orlandoterrorattack 3820.0 1769.5 5610.0 10
parisattacks 5015.0 3487.25 5266.5 10
peaceandreconciliation 5968.0 4840.75 6551.25 10
sanbernardinoshooting 3834.0 3811.25 3865.0 10
tunisiaattack2015 1682.0 1038.75 1835.75 10
turkeycoupattempt 6171.0 4859.25 6209.5 10
zikavirus 4550.0 4520.0 5580.0 3


In [244]:
for topic, group in sh.groupby('class'):
    group = group.sort_values(['scores'], ascending=False)[:20]
    print topic, group['order'].median(), group['order'].quantile(0.25), group['order'].quantile(0.75), group.shape[0]

bastilledaytruckattack 6124.0 4970.75 6204.5 20
berlinchristmasmarketattack 7490.0 4120.5 7518.5 15
brusselsattacks 5035.0 4653.0 5175.5 20
charliehebdoattack 217.0 75.75 2720.0 20
francetrainattack 2743.0 1382.0 5497.0 20
munichshooting 6240.0 5914.0 6256.0 17
orlandoterrorattack 3875.0 2185.75 5362.0 20
parisattacks 3532.0 3242.25 5091.0 20
peaceandreconciliation 5706.0 4840.0 6625.0 17
sanbernardinoshooting 3834.0 3494.5 4759.0 20
tunisiaattack2015 1672.0 836.5 1787.0 20
turkeycoupattempt 6136.5 4003.75 6221.0 20
zikavirus 4550.0 4520.0 5580.0 3


In [310]:
def keyword_match(group, topic):
    test_transformed_text.ix[group.index].str.split().map(
        lambda x: sum(
            [tfidf.idfs[dictionary.token2id[i]] for i in set(
                tpa[tpa.topics == topic].actual.values[0]
            ).intersection(x)]
        )
    )

In [341]:
N = 100
min_N = 5
train_test_data = {}
for topic, group in sh.groupby('class'):
    group = group.sort_values(['scores'], ascending=False)[:N]


#     if group.shape[0] < min_N:
#         train_test_data[topic] = sorted(group.index)
#         print topic, group.shape[0]
#         continue

#     minb = group['order'].quantile(0.25)
#     maxb = group['order'].quantile(0.75)
    
    keyword_weight = keyword_match(group, topic)
    group['kw'] = keyword_weight

#     group = group[group.order.between(minb, maxb)]
#     print topic, group.shape[0]

#     train_test_data[topic] = sorted(group.index)
    train_test_data[topic] = sorted(group.sort_values('kw', ascending=False)[:8].index)

In [308]:
set(tpa[tpa.topics == topic].actual.values[0])

{'zika'}

TestData_04490    15.66276
TestData_04550    15.66276
TestData_06610    15.66276
Name: bodyText, dtype: float64

In [254]:
sh[sh['class'] == 'berlinchristmasmarketattack']

,class,scores,order
TestData_01999,berlinchristmasmarketattack,0.602871,1999
TestData_07346,berlinchristmasmarketattack,0.610108,7346
TestData_03028,berlinchristmasmarketattack,0.625340,3028
TestData_04118,berlinchristmasmarketattack,0.630178,4118
TestData_04123,berlinchristmasmarketattack,0.667375,4123
TestData_07515,berlinchristmasmarketattack,0.678086,7515
TestData_07545,berlinchristmasmarketattack,0.685988,7545
TestData_04254,berlinchristmasmarketattack,0.695285,4254
TestData_07558,berlinchristmasmarketattack,0.741265,7558
TestData_07514,berlinchristmasmarketattack,0.789414,7514


In [252]:
train_test_data

{'bastilledaytruckattack': [u'TestData_05261',
  u'TestData_06106',
  u'TestData_06107',
  u'TestData_06110',
  u'TestData_06122',
  u'TestData_06126',
  u'TestData_06141',
  u'TestData_06155',
  u'TestData_06158',
  u'TestData_06171'],
 'berlinchristmasmarketattack': [u'TestData_04123',
  u'TestData_04254',
  u'TestData_07346',
  u'TestData_07490',
  u'TestData_07504',
  u'TestData_07514',
  u'TestData_07515'],
 'brusselsattacks': [u'TestData_05019',
  u'TestData_05025',
  u'TestData_05027',
  u'TestData_05028',
  u'TestData_05029',
  u'TestData_05041',
  u'TestData_05051',
  u'TestData_05078',
  u'TestData_05103',
  u'TestData_05126'],
 'charliehebdoattack': [u'TestData_00077',
  u'TestData_00121',
  u'TestData_00156',
  u'TestData_00172',
  u'TestData_00185',
  u'TestData_00249',
  u'TestData_01147',
  u'TestData_01162',
  u'TestData_01329',
  u'TestData_02540'],
 'francetrainattack': [u'TestData_01427',
  u'TestData_01640',
  u'TestData_02263',
  u'TestData_02264',
  u'TestData_022

In [323]:
train_test_data

{'bastilledaytruckattack': [u'TestData_01662',
  u'TestData_06106',
  u'TestData_06107',
  u'TestData_06122',
  u'TestData_06126',
  u'TestData_06155',
  u'TestData_06158',
  u'TestData_06171'],
 'berlinchristmasmarketattack': [u'TestData_03682',
  u'TestData_04254',
  u'TestData_07490',
  u'TestData_07504',
  u'TestData_07514',
  u'TestData_07522',
  u'TestData_07543',
  u'TestData_07558'],
 'brusselsattacks': [u'TestData_05025',
  u'TestData_05027',
  u'TestData_05029',
  u'TestData_05041',
  u'TestData_05103',
  u'TestData_05126',
  u'TestData_05324',
  u'TestData_07513'],
 'charliehebdoattack': [u'TestData_00047',
  u'TestData_00072',
  u'TestData_00121',
  u'TestData_00156',
  u'TestData_00172',
  u'TestData_01147',
  u'TestData_01162',
  u'TestData_04189'],
 'francetrainattack': [u'TestData_00058',
  u'TestData_01033',
  u'TestData_01247',
  u'TestData_02263',
  u'TestData_02264',
  u'TestData_02282',
  u'TestData_03204',
  u'TestData_05476'],
 'munichshooting': [u'TestData_04175

In [342]:
train_test_data

{'bastilledaytruckattack': [u'TestData_01662',
  u'TestData_06106',
  u'TestData_06107',
  u'TestData_06122',
  u'TestData_06126',
  u'TestData_06155',
  u'TestData_06158',
  u'TestData_06171'],
 'berlinchristmasmarketattack': [u'TestData_03682',
  u'TestData_04254',
  u'TestData_07490',
  u'TestData_07504',
  u'TestData_07514',
  u'TestData_07522',
  u'TestData_07543',
  u'TestData_07558'],
 'brusselsattacks': [u'TestData_05025',
  u'TestData_05027',
  u'TestData_05029',
  u'TestData_05041',
  u'TestData_05103',
  u'TestData_05126',
  u'TestData_05324',
  u'TestData_07513'],
 'charliehebdoattack': [u'TestData_00047',
  u'TestData_00072',
  u'TestData_00121',
  u'TestData_00156',
  u'TestData_00172',
  u'TestData_01147',
  u'TestData_01162',
  u'TestData_04189'],
 'francetrainattack': [u'TestData_00058',
  u'TestData_01033',
  u'TestData_01247',
  u'TestData_02263',
  u'TestData_02264',
  u'TestData_02282',
  u'TestData_03204',
  u'TestData_05476'],
 'munichshooting': [u'TestData_04175

## Use 7512 for Berlin Christmas market attack / Germany / Paris attacks / Brussels attacks / France / analysis

In [277]:
test_df.iloc[3398].bodyText # 19

u'Four days after 129 people were killed and 352 wounded in gun and bomb attacks in Paris, people around the world are trying to come to terms with what happened. Vigils and events have taken place in a show of solidarity with France. French Muslim leaders gathered outside the Bataclan concert hall on Sunday to honour the 89 people who died there; on Monday France held a minute\u2019s silence at noon, joined by many people in Britain; and events have been held around the world to remember the victims. But how have the terrorist attacks affected you? If you\u2019re Muslim, we want to hear your thoughts on how the Paris attacks have impacted you and your community. Fill in the form below - you do not have to answer every question - and we\u2019ll use a selection of responses in our reporting.'

In [284]:
# 'tfidf_wvec': [test_tfidf_wvec, unseen_topics2vec.tfidf_wv],
# 'tfidf_fvec': [test_tfidf_fvec, unseen_topics2vec.tfidf_fs],
# 'tfidf_lsi': [test_tfidf_lsi, unseen_topics2vec.tfidf_lsi],

sim = cosine_similarity(
    np.vstack(test_tfidf_wvec),
    np.vstack(unseen_topics2vec.tfidf_wv),
)
N = 5
for i, t in enumerate(unseen_topics2vec.index):
    idx = sim[:, i].argsort()[::-1][:N]
    v = sim[:, i][idx]
    print unseen_topics2vec.index[i], zip(idx[:N], np.round(v[:N], 1))


activism [(2265, 0.5), (4471, 0.5), (152, 0.5), (5445, 0.5), (6796, 0.5)]
bastilledaytruckattack [(6128, 0.80000000000000004), (6098, 0.69999999999999996), (6121, 0.69999999999999996), (6105, 0.69999999999999996), (6126, 0.69999999999999996)]
berlinchristmasmarketattack [(3681, 0.80000000000000004), (7491, 0.80000000000000004), (7521, 0.69999999999999996), (7503, 0.69999999999999996), (7489, 0.69999999999999996)]
brusselsattacks [(5125, 0.80000000000000004), (5019, 0.80000000000000004), (5046, 0.80000000000000004), (3515, 0.80000000000000004), (5102, 0.69999999999999996)]
charliehebdoattack [(71, 0.90000000000000002), (4276, 0.90000000000000002), (60, 0.90000000000000002), (4188, 0.90000000000000002), (120, 0.90000000000000002)]
francetrainattack [(3514, 0.69999999999999996), (1246, 0.69999999999999996), (3198, 0.69999999999999996), (2263, 0.69999999999999996), (6256, 0.69999999999999996)]
munichshooting [(6267, 0.80000000000000004), (6231, 0.80000000000000004), (6236, 0.69999999999999

In [272]:
v

array([ 0.85859394,  0.57481531,  0.56485035,  0.55066684,  0.5251468 ,
        0.45473479,  0.44471953,  0.44246537,  0.42221175,  0.42088665,
        0.4148486 ,  0.41344707,  0.4058871 ,  0.40490635,  0.39717574,
        0.39518468,  0.39454976,  0.3910347 ,  0.38950969,  0.38351464])

In [125]:
test_max_class_df.ix['TestData_04490']

tfidf_fvec    zikavirus
tfidf_wvec    zikavirus
tfidf_lsi         ebola
Name: TestData_04490, dtype: object

In [262]:
tpvec.index[41]

'ebola'

In [126]:
test_max_class_df.iloc[4489]

tfidf_fvec    zikavirus
tfidf_wvec    zikavirus
tfidf_lsi         ebola
Name: TestData_04490, dtype: object

In [78]:
test_max_class_df.iloc[4489]

tfidf_fvec    zikavirus
tfidf_wvec    zikavirus
tfidf_lsi         ebola
Name: TestData_04490, dtype: object

In [222]:
sim.argmax(axis=1)

array([151, 134,  29, ...,  72, 136, 136])

In [216]:
Counter(sim.argmax(axis=1))

Counter({0: 14,
         1: 38,
         2: 15,
         3: 12,
         4: 27,
         5: 35,
         6: 5,
         7: 9,
         8: 8,
         9: 32,
         10: 85,
         11: 201,
         12: 6,
         13: 16,
         14: 15,
         15: 2,
         16: 10,
         17: 8,
         18: 12,
         19: 38,
         20: 7,
         21: 16,
         22: 70,
         23: 11,
         24: 20,
         25: 8,
         26: 17,
         27: 44,
         28: 31,
         29: 18,
         30: 5,
         31: 3,
         32: 47,
         33: 30,
         34: 8,
         35: 9,
         36: 32,
         37: 68,
         38: 1,
         39: 3,
         40: 27,
         41: 38,
         42: 11,
         43: 56,
         44: 38,
         45: 19,
         46: 17,
         47: 5,
         48: 69,
         49: 15,
         50: 28,
         51: 2,
         52: 8,
         53: 254,
         54: 17,
         55: 25,
         56: 124,
         57: 26,
         58: 34,
         59: 8,
     

In [221]:
np.where(all_topics2vec.index.str.contains('bastille'))

(array([11]),)

In [259]:
tfidf.idfs[dictionary.token2id['virus']]

7.202270959611463

In [168]:
test_doc_tokens = pd.DataFrame({k: {dictionary.id2token[t]: c for t, c in dictionary.doc2bow(i)} for k, i in zip(test_transformed_text.index, test_transformed_text.str.split())})

In [178]:
test_doc_tokens = test_doc_tokens.T

In [217]:
test_doc_tokens[test_doc_tokens.zika > 0].shape

(10, 84860)

In [ ]:
def score_topic_doc_match(doc_tokens, topic_tokens):
    

In [84]:
unseen_topics_rank

{'activism': [array([6796, 5937,  396, 4906, 4471]),
  array([6796, 6624, 4906,  396, 2265]),
  array([7182, 4421,  170, 3121, 4062])],
 'bastilledaytruckattack': [array([6128,   78, 3188, 6135, 3210]),
  array([6128,   78, 3220, 3188, 3210]),
  array([5142, 3187, 3182, 4340, 7481])],
 'berlinchristmasmarketattack': [array([7491, 6818, 3681, 7489, 6267]),
  array([7491, 6267, 7489, 3681, 6818]),
  array([7489, 3681, 7535, 4253, 7513])],
 'brusselsattacks': [array([3300, 5008, 4458, 5080, 7056]),
  array([3300, 5008, 3554, 5019, 5028]),
  array([2062, 3620, 5028, 5113, 5052])],
 'charliehebdoattack': [array([3124,   45,   67, 4198,   49]),
  array([3124,   45,   49, 4198, 2539]),
  array([3182, 4198, 3873, 2062, 4961])],
 'francetrainattack': [array([3514, 2263, 5028, 5475, 6636]),
  array([3514, 2263, 5028, 5475, 1246]),
  array([6118, 3200, 7210, 2263, 3649])],
 'munichshooting': [array([6268, 6254, 6231, 6267, 6232]),
  array([4031, 6268, 6231, 2129, 6254]),
  array([6239, 6254, 6249

In [319]:
test_df.iloc[3514].bodyText # 19

u'French police have tweeted a picture of the third man they believe carried out the bomb attack on the Stade de France last week, and appealed for help in identifying him. The Police Nationale released the image on Sunday evening, nine days after the bombings, which were part of a series of simultaneous attacks across the French capital that killed 130 people. It followed police raids in which three people suspected of involvement in the attacks were killed. The man\u2019s image was accompanied by a message that read: \u201cThis individual is the third dead perpetrator of one of the attacks carried out on 13 November at the Stade de France.\u201d In Belgium, the prime minister, Charles Michel, said the terror alert would remain at the country\u2019s highest level for a sustained \u201cserious and imminent\u201d threat against the capital, Brussels. Michel said that, like the weekend, authorities fear a Paris-like attack, \u201ceven perhaps at several locations\u201d. He said that cons

In [253]:
test_df[test_df.bodyText.str.contains('Zika')].bodyText.iloc[2]

u'Violent protests at Trump rally in California With the tenor of the general election campaign now clearly defined, Hillary Clinton delivered a stinging rebuke to Donald Trump\u2019s claim to understand international affairs on Thursday. Trump, meanwhile, described Clinton\u2019s clear-headed foreign policy address \u2013 in which she described him as too unstable to be trusted with nuclear launch codes and warned he would take the country down a \u201ctruly dangerous path\u201d \u2013 as \u201cpathetic\u201d and said she should be in jail. Violence later erupted at a Trump rally in California. Reports from San Jose described anti-Trump demonstrators chasing \u2013 and in some cases punching and attacking \u2013 departing Trump supporters, some of whom appeared intent on provoking and fighting. Riot police were deployed to control the crowd. The mayor of San Jose, Sam Liccardo, blamed Trump. \u201cWe don\u2019t appreciate [anyone] utilizing campaign tactics of demagoguery,\u201d he sa

In [245]:
ssim = cosine_similarity(
    np.vstack(test_tfdifwvec)[4489],
    np.vstack(test_tfdifwvec),
)

/home/avsolatorio/ml-ai/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [250]:
ssim.argsort()[0][::-1][:20]

array([4489, 7543, 4760, 4881,  471, 5557,  373,  824, 4456, 4549,  352,
        469,  749, 1906, 1092, 6609, 2255,  950, 2888, 1512])

In [252]:
topics

['activism',
 'afghanistan',
 'aid',
 'algerianhostagecrisis',
 'alqaida',
 'alshabaab',
 'antiwar',
 'arabandmiddleeastprotests',
 'armstrade',
 'australianguncontrol',
 'australiansecurityandcounterterrorism',
 'bastilledaytruckattack',
 'belgium',
 'berlinchristmasmarketattack',
 'bigdata',
 'biometrics',
 'bokoharam',
 'bostonmarathonbombing',
 'britisharmy',
 'brusselsattacks',
 'cameroon',
 'carers',
 'charliehebdoattack',
 'chemicalweapons',
 'clusterbombs',
 'cobra',
 'conflictanddevelopment',
 'controversy',
 'criminaljustice',
 'cybercrime',
 'cyberwar',
 'darknet',
 'dataprotection',
 'debate',
 'defence',
 'deflation',
 'drones',
 'drugs',
 'drugspolicy',
 'drugstrade',
 'earthquakes',
 'ebola',
 'economy',
 'egypt',
 'encryption',
 'energy',
 'espionage',
 'ethics',
 'europeanarrestwarrant',
 'europeancourtofhumanrights',
 'events',
 'extradition',
 'famine',
 'farright',
 'firefighters',
 'forensicscience',
 'france',
 'francetrainattack',
 'freedomofspeech',
 'genevaconv